## Perl 6 on Jupyter

# FOSDEM 2018

Brian Duggan

github:bduggan &bull; irc:bdmatatu &bull; bduggan@matatu.org

![](files/img/promptworks-logo.png)

![](files/img/promptworks-logo.png)

We craft software for companies
that care about the details.

# Outline
* &rarr;**Jupyter**
* Architecture
* Console
* Autocompletion
* Notebooks: Basics
* Magics
* Comms (asynchronous)
* Notebooks: Fancier

## but first

If you have a laptop, you can go to

http://github.com/bduggan/p6-jupyter-kernel

and click "Launch Binder", to experiment during this talk.

### Jupyter
* iPython: 2001-present
* Jupyter: 2014-present
* Goals
 * Improve Read-Eval-Print-Loop (REPL) interface
 * "Rich" Interactive Shells
 * Visualizing Output
 * Serializing, Sharing, Replaying REPL sessions
 * Started with Julia, R, Python
 * Now over 40 languages, including Perl 6
 * ~ 500,000 "notebooks" on github
 * Popular in scientific community
 * Used for collaboration and research
 * Some integration with journals/publishers

# Outline
* ✓ Jupyter
* &rarr; **Architecture**
* Console
* Autocompletion
* Notebooks: Basics
* Magics
* Comms (asynchronous)
* Notebooks: Fancier

## Architecture
<img src='files/img/repl1-3.svg' width='40%'>
* Typical REPL flow, often a single process on a single computer.

#### Architecture
![](files/img/repl2.svg)
* Separate client, jupyter server, and kernel
* One server can support multiple clients, multiple kernels
* Kernel can be stopped, started
* Replay input to reproduce output

####  Architecture
![](files/img/notebook_components.png)
<small style='font-size:30%'>
source: http://jupyter.readthedocs.io/en/latest/architecture/how_jupyter_ipython_work.html
</small>

### Architecture
* Notebooks are independent; dedicated kernels.
* No security!
* Assumption: User controls the client and the server.
* Kernel may use arbitrary resources.
* Notebook file is JSON (`.ipynb` suffix).
* Contains serialized session.
* Static file, renders well on github and nbviewer.jupyter.org

# Outline
* ✓ Jupyter
* ✓ Architecture
* &rarr; **Console**
* Autocompletion
* Notebooks: Basics
* Magics
* Comms (asynchronous)
* Notebooks: Fancier

###  Console
Launch console client and server:

```
alias iperl6='jupyter-console --kernel=perl6'
```
![](files/img/welcome.png)

### Console
![](files/img/hello.png)
* Syntax highlighting
* `Out[*]`, `_*` have results, stdout in between
* No output for `Nil` (or a final print)

### Autocompletion
<img style='width:60%' src='files/img/autocomplete.png'>
* Lexical variables.
   * Searches `LEXICAL::.keys`

#### Autocompletion
* Autocomplete methods by typing `.` and `[tab]`
* `12.` will search within `Int`
* `12.f` will search ancestors in the type hierarchy
![](files/img/int-type.svg)

In [ ]:
12.^methods

* Autocompletion
  * Extra
  * `(` to find set operators (e.g. to type ` $a ∪ $b` instead of `$a (|) $b`)
  * `**` to type exponents (e.g. for `2⁷` instead of `2 ** 7`)
  * `atomic` to type atomic operators (for `$a ⚛= 10` instead of `atomic-assign($a,10)`)
  
<img style='width:60%;' src='files/img/set.png'>

# Outline
* ✓ Jupyter
* ✓ Architecture
* ✓ Console
* ✓ Autocompletion
* &rarr; ** Notebooks: Basics **
* Magics
* Comms (asynchronous)
* Notebooks: Fancier

### Notebooks
Launch web server and client:

     $ jupyter-notebook
     File → New Notebook → Perl 6

In [ ]:
<hello world>.join(' ') 

In [ ]:
2 ** 127 - 1

### Notebooks

Important commands:
* ctrl-enter -- execute cell
* alt-enter -- execute cell, add another
* `In[*]` prompt means it is running
* `In[2]` when done (for cell 2)
* Kernel → Restart & Run All
* Kernel → Restart & Clear Output

### Notebooks
* Output cell and stdout can have MIME types.
* SVG is recognized

In [ ]:

# SVG output is recognized
q:to/SVG/
  <svg height="100" width="100">
   <circle cx="50" cy="50" r="40"
           stroke="black" stroke-width="3"
           fill="red" />
  SVG
# or use SVG::Plot

# Outline
* ✓ Jupyter
* ✓ Architecture
* ✓ Console
* ✓ Autocompletion
* ✓ Notebooks: Basics
* &rarr; ** Magics **
* Comms (asynchronous)
* Notebooks: Fancier

## Magics
* First line of the cell, starts with `%%` or `#%`.
* `%% html`, `%% latex` : specify the MIME type of the output

In [ ]:
%% html
'<b>hello</b>, <i>world</i>';

In [ ]:
%% latex(equation)
'\frac{1}{2}'

In [ ]:
%% latex(equation)
q:to/LATEX/
  \frac{{ - b \pm \sqrt {b^2 - 4ac} }}{{2a}}
LATEX

### Magics
* `bash`. `run`

In [ ]:
%% bash
# Treat the cell contents as bash
echo 'my $distance = 12_0000;' > calc.p6
echo 'my $speed = 120;' >> calc.p6
echo 'my $time = $distance / $speed;' >> calc.p6 
cat calc.p6

In [ ]:
%% run calc.p6
say $distance;

### Magics
* `javascript`

In [ ]:
%% javascript
element.append($('<div>').text('hello, world'))

In [ ]:
%% javascript
// load chart.js
require.config({ paths: {
 chartjs: [
    "files/js/Chart.bundle",
    // "https://cdnjs.cloudflare.com/ajax/libs/Chart.js/2.7.1/Chart.bundle"
    ]
}});
require(['chartjs'])

In [ ]:
# data for chart.js
my $counts = {
  :type<bar>,
  data => {
    labels =>  ["😊".."😏"],
    datasets => [ {
        :label<count>,
         data => <13 19 3 5 10 3>,
     }, ]
  }
};

In [ ]:
%% html
use JSON::Fast;
qq:to/OUT/;
<canvas id="emojis" width="50" height="10"></canvas>
<script>
var canvas = document.getElementById("emojis");
var j = JSON.parse('{ to-json($counts, :!pretty) }');
new Chart(canvas.getContext('2d'), j);
</script>
OUT

# Outline
* ✓ Jupyter
* ✓ Architecture
* ✓ Console
* ✓ Autocompletion
* ✓ Notebooks: Basics
* ✓ Magics
* &rarr; ** Comms (asynchronous) **
* Notebooks: Fancier

### Comms
> The comm API is a symmetric, asynchronous, fire and forget style messaging API. It allows the programmer to send JSON-able blobs between the front-end and the back-end. 

<small style='font-size:50%'>
-- http://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Low%20Level.html
</small>

### Comms
* `$*JUPYTER.register-comm($name, $routine)`  : register a comm.
* `$routine(Channel :$in,Channel :$out,:$data)` : will be in a separate thread.

In [ ]:
$*JUPYTER.register-comm: 'counter', -> Channel :$out {
    react whenever Supply.interval(1) -> $i {
         $out.send($i);
         done if $i == 5;
  }
}

In [ ]:
%% javascript
var comm = Jupyter.notebook.kernel.comm_manager.new_comm('counter');
var el = $('<div>'); element.append(el);
comm.on_msg(function(msg) { el.html(msg.content.data); });

In [ ]:
$*JUPYTER.register-comm('adder', -> :$in, :$out {
      my atomicint $num = 0;
      start react whenever Supply.interval(1) -> $i {
         $out.send($num + $i);
         done if $i > 4;
      }
      start while $in.receive -> $i {
         $num ⚛= $i;
      }
   }
)

In [ ]:
%% javascript
var comm = Jupyter.notebook.kernel.comm_manager.new_comm('adder');
var el = $('<div>'); element.append(el);
var x = 10;
comm.on_msg(function(msg) {
   el.html(msg.content.data);
   comm.send(x);
   x = x + 10;
});

# Outline
* ✓ Jupyter
* ✓ Architecture
* ✓ Console
* ✓ Autocompletion 
* ✓ Notebooks: Basics
* ✓ Magics
* ✓ Comms (asynchronous)
* ** Notebooks: Fancier **

### Fancier Notebook

Let's use d3.js to visualize the fibonacci sequence.

We want to do computation on the server, and stream results to the client for visualization.

We'll send new values once a second, simulating a time-consuming calculation.

Fibonacci sequence: $ 1, 1, 2, 3, 5, 8, 13, ... $

Formula :
$$ F_1 = 1 $$
$$ F_2 = 1 $$
$$ F_n = F_{n-1} + F_{n-2} $$

Ratio of successive terms:

$$ \frac{1}{1}, \frac{2}{1}, \frac{3}{2}, \frac{5}{3}, \frac{8}{5}, \frac{13}{8}, ... \frac{F_n+1}{F_n} $$

Let's try to visualize and numerically verify that:

$$ \lim_{n\to\infty}\frac{F_n+1}{F_{n}} = \frac{1 + \sqrt{5} }{2}$$

![](files/img/spiral-2.png)

| |↗|↖|↙|↘|↗|...|
|---|---|---|---|---|---|---|
|fibonacci | 1 | 1 | 2 | 3 | 5 | ... |
|direction|1,1|-1,1|-1,-1|1,-1|1,1|...|
|diagonal|1,1|-1,1|-2,-2|3,-3|5,5|...|
|corner|1,1|0,2|-2,0|1,-3|6,2|...

| |↗|↖|↙|↘|↗|...|
|---|---|---|---|---|---|---|
|fibonacci | 1 | 1 | 2 | 3 | 5 | ... |
|direction|1,1|-1,1|-1,-1|1,-1|1,1|...|
|diagonal|1,1|-1,1|-2,-2|3,-3|5,5|...|
|corner|1,1|0,2|-2,0|1,-3|6,2|...

In [ ]:
my @fibonacci  = 1, 1, * + * ... ∞;
@fibonacci[^20];

| |↗|↖|↙|↘|↗|...|
|---|---|---|---|---|---|---|
|fibonacci | 1 | 1 | 2 | 3 | 5 | ... |
|direction|1,1|-1,1|-1,-1|1,-1|1,1|...|
|diagonal|1,1|-1,1|-2,-2|3,-3|5,5|...|
|corner|1,1|0,2|-2,0|1,-3|6,2|...

In [ ]:
my @directions = |(<1 1>, <-1 1>, <-1 -1>, <1 -1>) xx ∞;
@directions[^5]

| |↗|↖|↙|↘|↗|...|
|---|---|---|---|---|---|---|
|fibonacci | 1 | 1 | 2 | 3 | 5 | ... |
|direction|1,1|-1,1|-1,-1|1,-1|1,1|...|
|diagonal|1,1|-1,1|-2,-2|3,-3|5,5|...|
|corner|1,1|0,2|-2,0|1,-3|6,2|...

In [ ]:
my @diagonals  = @directions Z»*» @fibonacci;
@diagonals[^5]

* `Z»*»` is `Z` zip meta operator + `»»` hyper operator + `*` operator.

| |↗|↖|↙|↘|↗|...|
|---|---|---|---|---|---|---|
|fibonacci | 1 | 1 | 2 | 3 | 5 | ... |
|direction|1,1|-1,1|-1,-1|1,-1|1,1|...|
|diagonal|1,1|-1,1|-2,-2|3,-3|5,5|...|
|corner|1,1|0,2|-2,0|1,-3|6,2|...

In [ ]:
my @corners    = [\»+»] @diagonals;
@corners.unshift: (0,0);
@corners[^5]

* `[\»+»]` is `[\]` triangular reduce operator + `»»` hyper operator + `+` operator.

In [ ]:
# We'll also need rotations to help with our visualizations.
my @rotations  = |<0 90 180 270> xx ∞;
@rotations[^5]

Ratios

$$ \frac{1}{1}, \frac{2}{1}, \frac{3}{2}, \frac{5}{3}, \frac{8}{5}, \frac{13}{8}, ... \frac{F_n+1}{F_n} $$

In [ ]:
my @pairs  = @fibonacci.rotor(2 => -1);
@pairs[^5]

In [ ]:
my @ratios = @pairs.map: -> ($a,$b) { $b.FatRat / $a.FatRat }
@ratios[^5]

Accuracy

\begin{align}
\frac{F_n+1}{F_{n}} = r & \approx \frac{1 + \sqrt{5} }{2} \\
2r & \approx 1 + \sqrt{5} \\
(2r - 1)^2 & \approx 5 \\
(2r - 1)^2 - 5 & \approx 0
\end{align}

In [ ]:
my @accuracy = @ratios.map: -> \r { (2 * r - 1)² - 5 }
@accuracy[^5]

Construct a sequence of messages to send to the client.

In [ ]:
my @squares = ( [Z,] @corners, @fibonacci, @rotations, @ratios, @accuracy ).map: {
   %( corner   => .[0],
      height   => .[1],
      rotation => .[2],
      ratio    => .[3].Str,
      fraction => .[3].nude,
      accuracy => .[4].Str
    )
}
@squares[0]

In [ ]:
$*JUPYTER.register-comm('golden-ratio',
-> :$out {
      start react whenever Supply.interval(0.75) -> $i {
         $out.send(@squares[$i]);
         done if $i > 30;
      }
   }
)

Meanwhile, on the client side...

In [ ]:
%% javascript
require.config( { paths: {
   'd3' : [   
      // 'https://d3js.org/d3.v4.min'
      'files/js/d3.v4.min',
   ]
   }});
require(['d3'], function(d3) { window.d3 = d3 } )

In [ ]:
%% javascript
// window.add_svg = function ...
// window.add_square = function ..
// window.zoom_out = function ..
// window.display_ratio = function...
$.get('files/helpers.js')

In [ ]:
%% javascript
var comm = Jupyter.notebook.kernel.comm_manager.new_comm('golden-ratio');
var svg = add_svg(this.element);
comm.on_msg(function(msg) {
   var square = msg.content.data;
   zoom_out(svg,square); 
   add_square(svg, square);
   display_ratio(square);
})

### Conclusions

* Many Perl 6 features fit well with Jupyter
* Unicode support
* Rational arithmetic
* Infinite sequences
* Asynchronous primitives

### Future Work and Open Questions
* Documentation browser
* Support more MIME types
* Widgets (based on comms); generate javascript
* What features belong in the built-in REPL?

### Thank you!
```
Also, thanks:
Earlier iterations: Timo Paulssen, arnsholt
Docker: Suman Khanal
Javascript: Tom Chandler, Ryan Hinkel
```
![](files/img/promptworks-logo.png)
